In [1]:
import math
import warnings
import numpy as np
import pandas as pd

from tqdm import tqdm
from matplotlib import pyplot as plt


warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
appa1 = pd.read_csv("data/appa1.csv")
appa1 = appa1.drop(columns='Unnamed: 0')

### FBK original dataset

| ts        | The time the data was taken (date time)                                                                      |
|-----------|--------------------------------------------------------------------------------------------------------------|
| Rs1 - Rs8 | Resistance value of all 8 sensors, usually paired the same way (1,2) (3,4)...                                |
| Rh1 - Rh8 | Resistance value of the heaters, ideally should stay stable or correlated to tempurature                     |
| Vh1 - Vh8 | Voltage of the heater                                                                                        |
| T         | Temperature in C                                                                                             |
| RH        | Relative Humidity                                                                                            |
| P         | Air pressure                                                                                                 |
| BME       | Volatile Organic Compounds (VOC), from the BME280 sensor. Likely unrelated but added because its extra data! |

In [3]:
appa1.at[381849, "Time"] = pd.to_datetime("Tue Jan 12 2021 15:39:26 GMT+0000")
+ pd.to_timedelta(1, unit="min")

Timedelta('0 days 00:01:00')

In [4]:
appa1 = appa1.rename(columns = {"ts": "Time", "T": "Temperature", "RH": "Relative_Humidity", "P": "Pressure", "T": "Temperature", "T": "Temperature", "BME": "VOC"})

# interpolate NaN values
for i, row in tqdm(appa1.iterrows(), total=len(appa1)):
    if type(row.Time) != str:
        print(i, appa1.iloc[i-1].Time, appa1.iloc[i].Time)
        appa1.at[i, "Time"] = pd.to_datetime(" ".join(appa1.iloc[i-1].Time.split(" ")[:6])) + pd.to_timedelta(1, unit="m")

  0%|          | 0/1183842 [00:00<?, ?it/s]

: 

: 

In [ ]:
appa1.Time = pd.to_datetime(
    appa1.Time.map(lambda x:  " ".join(x.split(" ")[:6]) if type(x) == str else x),
    utc=True)

In [ ]:
appa1 = appa1.set_index("Time")

In [ ]:
# redundant can just do 
appa1_hr = pd.DataFrame(columns = ['Time', 'Rs1', 'Rs2', 'Rs3', 'Rs4', 'Rs5', 'Rs6', 'Rs7', 'Rs8', 'Rh1',
       'Rh2', 'Rh3', 'Rh4', 'Rh5', 'Rh6', 'Rh7', 'Rh8', 'Vh1', 'Vh2', 'Vh3',
       'Vh4', 'Vh5', 'Vh6', 'Vh7', 'Vh8', 'Temperature', 'Relative_Humidity',
       'Pressure', 'VOC'])

for i in tqdm(range(0, len(appa1)-60, 60), total=int(len(appa1)-60)/60):
       row = appa1.iloc[i:i+60].mean(axis=0)
       temp = [appa1.iloc[i].Time]
       temp[1:] = row
       appa1_hr.loc[len(appa1_hr.index)] = temp

100%|█████████▉| 19722/19729.7 [02:02<00:00, 150.70it/s]/usr/local/anaconda3/lib/python3.9/site-packages/tqdm/std.py:533: TqdmWarning: clamping frac to range [0, 1]
  full_bar = Bar(frac,
100%|██████████| 19730/19729.7 [02:02<00:00, 161.20it/s]


In [ ]:
appa1_hr.to_csv("exports/appa1_hr.csv")
appa1_gt = pd.read_csv("exports/appa1_gt_clean_withCO.csv")
appa1_marco = pd.read_csv("appa1_gt.csv")
appa1_hr['time'] = appa1_marco['Time']

#appa1_hr.time = pd.to_datetime(appa1_hr.time)
#appa1_hr.time = appa1_hr.time.dt.tz_localize(None)

In [ ]:
for i, row in tqdm(appa1_hr.iterrows(), total = len(appa1_hr)):
    appa1_hr.at[i, "time"] = row.Time.replace(microsecond=0, second=0, minute=0)

# appa1_hr.Time = appa1_hr.Time.dt.round('60min')

100%|██████████| 19730/19730 [00:00<00:00, 25288.23it/s]


In [ ]:
appa1_hr = appa1_hr.rename(columns = {"Time": "time", "time":"Time"})

In [ ]:
appa1_hr.Time = pd.to_datetime(appa1_hr.Time)
appa1_hr.Time = appa1_hr.Time.dt.tz_localize(None)

In [ ]:
appa1_gt.Time = pd.to_datetime(appa1_gt.Time)
appa1_hr.merge(appa1_gt, how='left', on="Time")
appa1_hr.iloc[380]

In [ ]:
#6380
#3937
#17641

appa1_hr = appa1_hr.iloc[6380:]
print(len(appa1_hr))

13350


In [ ]:
appa1_gt = appa1_gt.iloc[3864:17569]
print(len(appa1_gt))

13705


In [ ]:
appa1_clean = appa1[appa1.Time >= '2021-01-14']
appa1_first = appa1_clean[appa1_clean.Time <= '2021-04-29']
appa1_second = appa1_clean[(appa1_clean.Time > '2021-04-29') & (appa1_clean.Time <= '2021-6-30')]
appa1_third = appa1_clean[(appa1_clean.Time > '2021-06-30') & (appa1_clean.Time <= '2022-1-14')]
appa1_fourth = appa1_clean[appa1_clean.Time > '2022-1-14']

In [ ]:
appa1_second = appa1_second.iloc[:, [0,
 1,
 2,
 5, #
 6, #
 3, #
 4, #
 7,
 8,

 9,
 10,
 13, #
 14, #
 11, #
 12, #
 15,
 16,

 17,
 18,
 21, #
 22, #
 19, #
 20, #
 23,
 24,
 25,
 26,
 27,
 28]]

appa1_third = appa1_third.iloc[:, [0,
5,6,1,2,3,4,7,8,

13,14,9,10,11,12,15,16,

 21,22,17,18,19,20,23,24,

 25,
 26,
 27,
 28]]

appa1_fourth = appa1_fourth.iloc[:, [0,
5,6,1,2,3,4,7,8,

13,14,9,10,11,12,15,16,

 21,22,17,18,19,20,23,24,

 25,
 26,
 27,
 28]]

In [ ]:
appa_clean = pd.DataFrame(
       np.vstack([appa1_first, appa1_second, appa1_third, appa1_fourth]),
       columns=[
              "Time", 

              "ZnOR_1",
              "ZnOR_2",
              "LaFeO3_1",
              "LaFeO3_2",
              "WO3_1",
              "WO3_2", 
              "to_del", 
              "to_del2",

              "ZnOR_heatR_1",
              "ZnOR_heatR_2",
              "LaFeO3_heatR_1",
              "LaFeO3_heatR_2",
              "WO3_heatR_1",
              "WO3_heatR_2", 
              "to_de3", 
              "to_del1",

              "ZnOR_heatV_1",
              "ZnOR_heatV_2",
              "LaFeO3_heatV_1",
              "LaFeO3_heatV_2",
              "WO3_heatV_1",
              "WO3_heatV_2", 
              "to_deal", 
              "to_de2",

              'Temperature', 
              'Relative_Humidity',
              'Pressure', 
              'VOC'
       ]
       )

In [ ]:
appa_clean = appa_clean[[
              "Time", 

              "ZnOR_1",
              "ZnOR_2",
              "LaFeO3_1",
              "LaFeO3_2",
              "WO3_1",
              "WO3_2", 

              "ZnOR_heatR_1",
              "ZnOR_heatR_2",
              "LaFeO3_heatR_1",
              "LaFeO3_heatR_2",
              "WO3_heatR_1",
              "WO3_heatR_2", 

              "ZnOR_heatV_1",
              "ZnOR_heatV_2",
              "LaFeO3_heatV_1",
              "LaFeO3_heatV_2",
              "WO3_heatV_1",
              "WO3_heatV_2", 

              'Temperature', 
              'Relative_Humidity',
              'Pressure', 
              'VOC'
       ]]

In [ ]:
appa_clean = appa_clean.set_index("Time")

In [ ]:
appa_clean["ZnOR_1_Age"] = 0
appa_clean["ZnOR_2_Age"] = 0
appa_clean["LaFeO3_1_Age"] = 0
appa_clean["LaFeO3_2_Age"] = 0
appa_clean["WO3_1_Age"] = 0
appa_clean["WO3_2_Age"] = 0

In [ ]:
appa_clean.index = pd.to_datetime(appa_clean.index)

In [ ]:
def norm(data):
    return (data)/(max(data))

appa_clean.loc[(appa_clean.index < "2021-06-30"), "ZnOR_2"] = norm(
    appa_clean[(appa_clean.index < "2021-06-30")].ZnOR_1) * max(
        appa_clean[(appa_clean.index > "2021-06-30")].ZnOR_2)

In [ ]:
appa_hr = pd.read_csv("data/appa1_hrgroup.csv")
appa_hr = appa_hr.drop(["Unnamed: 0"], axis=1)
appa_clean = appa_hr
appa_clean.Time = pd.to_datetime(appa_clean.Time)
appa_clean = appa_clean.set_index("Time")

In [ ]:
"""
appa1_clean = appa1[appa1.Time >= '2021-01-14']
appa1_first = appa1_clean[appa1_clean.Time <= '2021-04-29']
appa1_second = appa1_clean[(appa1_clean.Time > '2021-04-29') & (appa1_clean.Time <= '2021-6-30')]
appa1_third = appa1_clean[(appa1_clean.Time > '2021-06-30') & (appa1_clean.Time <= '2022-1-14')]
appa1_fourth = appa1_clean[appa1_clean.Time > '2022-1-14']


appa_clean["ZnOR_1_Age"] = 0
appa_clean["ZnOR_2_Age"] = 0
appa_clean["LaFeO3_1_Age"] = 0
appa_clean["LaFeO3_2_Age"] = 0
appa_clean["WO3_1_Age"] = 0
appa_clean["WO3_2_Age"] = 0
"""

for i,row in appa_clean.iterrows():
    age = i.timestamp() - pd.Timestamp("2021-01-14").timestamp()

    appa_clean.loc[i,"ZnOR_1_Age"] = age
    appa_clean.loc[i,"ZnOR_2_Age"] = age
    appa_clean.loc[i,"LaFeO3_1_Age"] = age
    appa_clean.loc[i,"LaFeO3_2_Age"] = age
    appa_clean.loc[i,"WO3_1_Age"] = age
    appa_clean.loc[i,"WO3_2_Age"] = age
for i,row in appa_clean[appa_clean.index > "2021-4-29"].iterrows():
    age = i.timestamp() - pd.Timestamp("2021-4-29").timestamp()

    appa_clean.loc[i,"WO3_1_Age"] = age
    appa_clean.loc[i,"WO3_2_Age"] = age

In [ ]:
appa_clean["sin_hour"] = appa_clean["Time"].dt.hour.apply(
    lambda x: math.sin(x) * 2 * (math.pi / 24)
)
appa_clean["cos_hour"] = appa_clean["Time"].dt.hour.apply(
    lambda x: math.cos(x) * 2 * (math.pi / 24)
)

appa_clean["sin_weekday"] = appa_clean["Time"].dt.weekday.apply(
    lambda x: math.sin(x) * 2 * (math.pi / 7)
)
appa_clean["cos_weekday"] = appa_clean["Time"].dt.weekday.apply(
    lambda x: math.cos(x) * 2 * (math.pi / 7)
)

appa_clean["sin_month"] = appa_clean["Time"].dt.month.apply(
    lambda x: math.sin(x) * 2 * (math.pi / 12)
)

appa_clean["cos_month"] = appa_clean["Time"].dt.month.apply(
    lambda x: math.cos(x) * 2 * (math.pi / 12)
)

appa_clean["sin_ordate"] = appa_clean["Time"].apply(
    lambda x: math.sin(x.toordinal()) * 2 * (math.pi / 366)
)
appa_clean["cos_ordate"] = appa_clean["Time"].apply(
    lambda x: math.cos(x.toordinal()) * 2 * (math.pi / 366)
)

appa_clean["year"] = appa_clean["Time"].dt.year

appa_clean.to_csv("exports/appa_hrgroup_time.csv")